### Mount Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Format Dataset

In [ ]:
import json

DATA_PATH = "/content/drive/MyDrive/Fiap/trn_finetune.jsonl"
OUTPUT_PATH_DATASET = "/content/drive/MyDrive/Fiap/formatted_trn.json"

def format_dataset_into_model_input(data):
    """
    Função corrigida para extrair 'prompt' e 'completion' corretamente.
    """
    prompt = data.get("prompt", "")
    completion = data.get("completion", "")

    instruction = "Generate a description for the following item."

    try:
        # Extrai o texto entre "Question:" e "Answer:" para ser o "input"
        input_text = prompt.split("Question:")[1].split("Answer:")[0].strip()
    except IndexError:
        input_text = ""

    # A resposta ("output") é sempre o campo "completion".
    response = completion.strip()

    return instruction, input_text, response

# Process the dataset line by line to avoid memory issues
with open(DATA_PATH, 'r', encoding='utf-8') as input_file, \
     open(OUTPUT_PATH_DATASET, 'w', encoding='utf-8') as output_file:

    for line in input_file:
        try:
            item = json.loads(line)
            instruction, input_text, response = format_dataset_into_model_input(item)

            # Adiciona uma verificação para não salvar linhas com output vazio
            if response:
                formatted_item = {
                    "instruction": instruction,
                    "input": input_text,
                    "output": response
                }
                output_file.write(json.dumps(formatted_item, ensure_ascii=False) + '\n')

        except json.JSONDecodeError as e:
            print(f"Skipping invalid JSON line: {line.strip()} - Error: {e}")
        except Exception as e:
            print(f"An error occurred processing line: {line.strip()} - Error: {e}")

print(f"Dataset salvo em {OUTPUT_PATH_DATASET}")

Dataset salvo em /content/drive/MyDrive/Fiap/formatted_trn.json


### Install Dependencies

In [ ]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes
!pip install transformers datasets

### Setup and Imports

In [ ]:
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
import json
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments


max_seq_length = 2048
dtype = None
load_in_4bit = True
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
]

### Load Model and Tokenizer

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

### Configure LoRA

In [5]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",

    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2025.9.9 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


### Format Prompts and Load Dataset

In [ ]:
import json
OUTPUT_PATH_DATASET = "/content/drive/MyDrive/Fiap/formatted_trn.json"


alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):

        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass


dataset = load_dataset("json", data_files=OUTPUT_PATH_DATASET, split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

### Setup SFTTrainer

In [ ]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

### Train Model

In [8]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,390,403 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: renatotapenna (renatotapenna-fiap) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: Detected [openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.745200
2,2.910100
3,2.611700
4,2.652600
5,2.335800
6,2.196800
7,2.193800
8,2.167400
9,2.097500
10,1.730900


### Run Inference

In [15]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Generate a description for the following item",
        "The Book of Revelation", # input
        "",
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Generate a description for the following item

### Input:
The Book of Revelation

### Response:
The Book of Revelation is a book of the Bible that is filled with mystery and intrigue. It is a book that is often misunderstood, but also a book that is filled with hope. In this book, author David A. deSilva offers an introduction to the book of Revelation that is both accessible and informative. He provides a helpful overview of the book, its context, and its themes, as well as a detailed commentary on each chapter. This book is a great resource for anyone who wants to learn more about the book of Revelation.<|end_of_text|>


### Save Model and Tokenizer

In [ ]:
model.save_pretrained("/content/drive/MyDrive/Fiap/lora_model") # Local saving
tokenizer.save_pretrained("/content/drive/MyDrive/Fiap/lora_model")

### Load Saved Model (fine-tuning)

In [ ]:
from unsloth import FastLanguageModel
from transformers import TextStreamer
import torch

# Define the path where you saved your model and tokenizer
SAVED_MODEL_PATH = "/content/drive/MyDrive/Fiap/lora_model"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length, # Use the same max_seq_length as during training
    dtype = dtype, # Use the same dtype as during training
    load_in_4bit = load_in_4bit, # Use the same load_in_4bit as during training
    device_map="auto", # Keep this to load the model onto the GPU
)

# Configure the model for inference
FastLanguageModel.for_inference(model)

# Define your input prompt
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

### Run Inference with Loaded Model (fine-tuning)

In [ ]:
# This cell can be run repeatedly with different inputs without reloading the model

item_to_describe = "The Prophet" # Change this input for different inferences

inputs = tokenizer(
[
    alpaca_prompt.format(
        "Generate a description for the following item",
        item_to_describe, # input
        "",
    )
], return_tensors = "pt").to("cuda")

# Run inference
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Generate a description for the following item

### Input:
The Prophet

### Response:
A prophet is a person who is regarded as an inspired teacher or proclaimer of truth. A prophet speaks for God as his/her authority. A prophet is someone who has a special relationship with God and is able to communicate God's message to others.<|end_of_text|>


### Load Base Model

In [ ]:
# Load the base model
model_base, tokenizer_base = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    device_map = "cuda", # Explicitly set device_map to cuda
)

# Define the alpaca_prompt
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

from transformers import TextStreamer

# Configure the base model for inference
FastLanguageModel.for_inference(model_base)

#Run Inference with Base Model

In [ ]:
# This cell can be run repeatedly with different inputs for the base model

item_to_describe_base = "Mog's Kittens" # Change this input for different inferences with the base model

# Prepare input for inference
inputs_base = tokenizer_base(
[
    alpaca_prompt.format(
        "Generate a description for the following item",
        item_to_describe_base, # input
        "",
    )
], return_tensors = "pt").to("cuda")

# Run inference with the base model
text_streamer_base = TextStreamer(tokenizer_base)
_ = model_base.generate(**inputs_base, streamer = text_streamer_base, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Generate a description for the following item

### Input:
Mog's Kittens

### Response:
The Mog's Kittens is a type of cat that is known for its distinctive appearance. It has a short, stocky body, a round face, and large, round eyes. The Mog's Kittens is also known for its playful and affectionate personality, and it is a popular choice as a pet.<|end_of_text|>
